In [3]:
from typing import List, Optional, Dict, Any, Literal, Annotated
from datetime import datetime
from enum import Enum
from pydantic import BaseModel, Field, field_validator, model_validator, ConfigDict, StringConstraints, conint, confloat
from typing_extensions import Self
from trails_openrouter_ollama import OpenRouterChatModel

In [9]:
from typing import List, Optional, Dict, Any, Literal, Annotated
from datetime import datetime
from enum import Enum
from pydantic import BaseModel, Field, field_validator, model_validator, ConfigDict, StringConstraints, conint, confloat
from typing_extensions import Self
from trails_openrouter_ollama import OpenRouterChatModel  # Import your custom model


# Define complex nested Pydantic models using Pydantic v2

class Priority(str, Enum):
    """Enumeration for task priority levels"""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"


class Address(BaseModel):
    """Address model with validation using Pydantic v2"""
    model_config = ConfigDict(str_strip_whitespace=True)

    street: str = Field(..., description="Street address")
    city: str = Field(..., description="City name")
    state: Annotated[str, StringConstraints(min_length=2, max_length=2)] = Field(..., description="2-letter state code")
    zip_code: Annotated[str, StringConstraints(pattern=r"^\d{5}(-\d{4})?$")] = Field(..., description="ZIP code")
    country: str = Field(default="USA", description="Country")

    @field_validator('state')
    @classmethod
    def validate_state(cls, v: str) -> str:
        return v.upper()


class ContactInfo(BaseModel):
    """Contact information with multiple fields using Pydantic v2"""
    email: Annotated[str, StringConstraints(pattern=r"^[\w\.-]+@[\w\.-]+\.\w+$")] = Field(...)
    phone: Optional[Annotated[str, StringConstraints(pattern=r"^\+?1?\d{10,14}$")]] = None
    preferred_contact: Literal["email", "phone", "both"] = "email"


class Person(BaseModel):
    """Person model with nested models using Pydantic v2"""
    first_name: Annotated[str, StringConstraints(min_length=1, max_length=50)] = Field(...)
    last_name: Annotated[str, StringConstraints(min_length=1, max_length=50)] = Field(...)
    age: conint(ge=0, le=150) = Field(..., description="Person's age")
    address: Address
    contact: ContactInfo
    nicknames: List[str] = Field(default_factory=list, max_length=5)
    metadata: Dict[str, Any] = Field(default_factory=dict)


class SimplePerson(BaseModel):
    """Simplified Person model for tasks"""
    first_name: str = Field(..., description="First name")
    last_name: str = Field(..., description="Last name")
    email: str = Field(..., description="Email address")


class Task(BaseModel):
    """Task model with various field types using Pydantic v2"""
    model_config = ConfigDict(validate_assignment=True)

    id: int
    title: Annotated[str, StringConstraints(min_length=1, max_length=200)] = Field(...)
    description: Optional[str] = None
    priority: Priority
    assigned_to: Optional[SimplePerson] = None  # Using SimplePerson instead of Person
    due_date: Optional[str] = Field(None, description="Due date in ISO format")
    tags: List[str] = Field(default_factory=list)
    completed: bool = False
    subtasks: List['Task'] = Field(default_factory=list, description="Nested subtasks")


class Project(BaseModel):
    """Complex project model with multiple nested structures using Pydantic v2"""
    model_config = ConfigDict(validate_assignment=True, str_strip_whitespace=True)

    name: str = Field(..., description="Project name")
    description: str = Field(..., description="Detailed project description")
    start_date: str = Field(..., description="Project start date in ISO format")
    end_date: Optional[str] = Field(None, description="Project end date in ISO format")
    budget: confloat(gt=0) = Field(..., description="Project budget in USD")
    team_members: List[Person] = Field(..., min_length=1, description="Project team members")
    tasks: List[Task] = Field(default_factory=list, description="Project tasks")
    project_manager: Person = Field(..., description="Project manager details")
    status: Literal["planning", "in_progress", "on_hold", "completed", "cancelled"] = "planning"
    milestones: Dict[str, str] = Field(default_factory=dict, description="Key milestones with dates")
    risk_factors: List[str] = Field(default_factory=list, max_length=10)
    success_metrics: Dict[str, float] = Field(default_factory=dict)


# Company structure with multiple departments
class Department(BaseModel):
    """Department within a company using Pydantic v2"""
    name: str
    head: Person
    employees: List[Person]
    budget: float
    projects: List[Project]


class Company(BaseModel):
    """Company with complex nested structure using Pydantic v2"""
    model_config = ConfigDict(validate_assignment=True)

    name: str = Field(..., description="Company name")
    founded_year: conint(ge=1800, le=2024) = Field(..., description="Year company was founded")
    headquarters: Address
    departments: List[Department]
    total_employees: conint(ge=1) = Field(..., description="Total number of employees")
    annual_revenue: Optional[float] = None
    is_public: bool = False
    stock_symbol: Optional[str] = None

    @model_validator(mode='after')
    def validate_stock_symbol(self) -> Self:
        """Validate stock symbol based on public status"""
        if self.is_public and not self.stock_symbol:
            raise ValueError("Public companies must have a stock symbol")
        if self.stock_symbol and not self.is_public:
            raise ValueError("Only public companies can have stock symbols")
        if self.stock_symbol:
            self.stock_symbol = self.stock_symbol.upper()
        return self


# Additional complex models for testing Pydantic v2 features

class GeoLocation(BaseModel):
    """Geographic location with validation"""
    latitude: confloat(ge=-90, le=90) = Field(..., description="Latitude")
    longitude: confloat(ge=-180, le=180) = Field(..., description="Longitude")
    altitude: Optional[float] = Field(None, description="Altitude in meters")


class DateRange(BaseModel):
    """Date range with validation using Pydantic v2"""
    start_date: str = Field(..., description="Start date in ISO format")
    end_date: str = Field(..., description="End date in ISO format")

    @model_validator(mode='after')
    def validate_date_range(self) -> Self:
        """Ensure end date is after start date"""
        if self.start_date and self.end_date:
            if self.start_date > self.end_date:
                raise ValueError("End date must be after start date")
        return self


class ProductReview(BaseModel):
    """Product review with computed fields (Pydantic v2 feature)"""
    model_config = ConfigDict(validate_assignment=True)

    product_name: str
    reviewer: Person
    rating: conint(ge=1, le=5) = Field(..., description="Rating from 1 to 5")
    title: Annotated[str, StringConstraints(max_length=100)] = Field(...)
    review_text: Annotated[str, StringConstraints(min_length=10, max_length=5000)] = Field(...)
    verified_purchase: bool = False
    helpful_count: conint(ge=0) = Field(default=0)
    total_votes: conint(ge=0) = Field(default=0)
    images: List[str] = Field(default_factory=list, max_length=10, description="Image URLs")
    location: Optional[GeoLocation] = None
    date_range: Optional[DateRange] = None

    @field_validator('images')
    @classmethod
    def validate_images(cls, v: List[str]) -> List[str]:
        """Validate that all images are valid URLs"""
        for url in v:
            if not url.startswith(('http://', 'https://')):
                raise ValueError(f"Invalid image URL: {url}")
        return v


# Initialize the model
def get_chat_model():
    """Initialize OpenRouterChatModel"""
    return OpenRouterChatModel(
        model="openai/gpt-oss-120b",  # or another model that supports function calling
        temperature=0.1,  # Lower temperature for more consistent structured output
        max_tokens=2000
    )


def test_simple_nested_model():
    """Test with a Person model containing nested Address and ContactInfo"""
    print("\n" + "="*50)
    print("Test 1: Simple Nested Model (Person)")
    print("="*50)

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(Person)

    result = structured_llm.invoke(
        "Generate details for a software engineer named John Smith, age 32, "
        "living in San Francisco, CA 94105 at 123 Market St. "
        "His email is john.smith@techcorp.com and phone is +14155551234."
    )

    print(f"Type: {type(result)}")
    print(f"Name: {result.first_name} {result.last_name}")
    print(f"Age: {result.age}")
    print(f"Address: {result.address.street}, {result.address.city}, {result.address.state}")
    print(f"Contact: {result.contact.email}")
    return result


def test_task_with_enum():
    """Test Task model with enum and optional fields"""
    print("\n" + "="*50)
    print("Test 2: Model with Enum and Optional Fields (Task)")
    print("="*50)

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(Task)

    result = structured_llm.invoke(
        "Create a high-priority task with ID 1 for updating the user authentication system. "
        "Title: 'Update Authentication System'. "
        "Assign to: Jane Doe (first_name: Jane, last_name: Doe, email: jane.doe@example.com). "
        "Due date: 2024-12-31. "
        "Create 3 subtasks: "
        "1) ID 101: 'Update password hashing' with high priority, "
        "2) ID 102: 'Add 2FA support' with critical priority, "
        "3) ID 103: 'Add audit logs' with medium priority."
    )

    print(f"Task ID: {result.id}")
    print(f"Title: {result.title}")
    print(f"Priority: {result.priority.value}")
    if result.assigned_to:
        print(f"Assigned to: {result.assigned_to.first_name} {result.assigned_to.last_name} ({result.assigned_to.email})")
    else:
        print("Assigned to: Unassigned")
    print(f"Subtasks count: {len(result.subtasks)}")
    for i, subtask in enumerate(result.subtasks, 1):
        print(f"  Subtask {i}: {subtask.title} (Priority: {subtask.priority.value})")
    return result


def test_complex_project():
    """Test complex Project model with multiple nested structures"""
    print("\n" + "="*50)
    print("Test 3: Complex Nested Model (Project)")
    print("="*50)

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(Project)

    result = structured_llm.invoke(
        "Create a project plan for developing a mobile app. Project name: 'SmartHealth App'. "
        "Budget: $500,000. Start date: 2024-01-15. "
        "Team: 3 developers (Alice Johnson, Bob Chen, Carol Williams), all from San Francisco. "
        "Project manager: David Brown from New York. "
        "Include 3 main tasks: Design UI/UX (high priority), Develop backend (critical), Testing (medium). "
        "Key milestones: Design Complete (2024-02-15), Beta Release (2024-04-01). "
        "Risk factors: Timeline constraints, Third-party API reliability."
    )

    print(f"Project: {result.name}")
    print(f"Budget: ${result.budget:,.2f}")
    print(f"Status: {result.status}")
    print(f"Team size: {len(result.team_members)}")
    print(f"Team members: {', '.join([m.first_name + ' ' + m.last_name for m in result.team_members])}")
    print(f"Project Manager: {result.project_manager.first_name} {result.project_manager.last_name}")
    print(f"Tasks: {len(result.tasks)}")
    print(f"Milestones: {result.milestones}")
    print(f"Risk factors: {', '.join(result.risk_factors)}")
    return result


def test_company_structure():
    """Test the most complex model: Company with departments and projects"""
    print("\n" + "="*50)
    print("Test 4: Very Complex Nested Model (Company)")
    print("="*50)

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(Company)

    result = structured_llm.invoke(
        "Create a tech company profile: 'TechCorp Inc', founded in 2010, "
        "headquarters at 1 Tech Plaza, Palo Alto, CA 94301. "
        "Public company (TECH). 500 employees, $50M annual revenue. "
        "Two departments: "
        "1) Engineering dept with head Sarah Lee and 2 employees working on 1 AI project "
        "2) Marketing dept with head Mike Ross and 1 employee working on 1 branding project. "
        "Keep it realistic but concise."
    )

    print(f"Company: {result.name}")
    print(f"Founded: {result.founded_year}")
    print(f"Public: {result.is_public} ({result.stock_symbol if result.stock_symbol else 'N/A'})")
    print(f"HQ: {result.headquarters.city}, {result.headquarters.state}")
    print(f"Departments: {len(result.departments)}")
    for dept in result.departments:
        print(f"  - {dept.name}: {len(dept.employees)} employees, {len(dept.projects)} projects")
    return result


def test_pydantic_v2_features():
    """Test Pydantic v2 specific features"""
    print("\n" + "="*50)
    print("Test 5: Pydantic v2 Features (ProductReview)")
    print("="*50)

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(ProductReview)

    result = structured_llm.invoke(
        "Create a product review for 'Wireless Headphones Pro' by John Miller, age 28, "
        "from Austin, TX. 5-star rating. Title: 'Best headphones ever!'. "
        "Review: 'These headphones have amazing sound quality and the battery lasts forever. "
        "The noise cancellation is perfect for my daily commute. Highly recommended!' "
        "Verified purchase. 45 people found it helpful out of 50 total votes. "
        "Include 2 image URLs (use example URLs like https://example.com/image1.jpg). "
        "Location: latitude 30.2672, longitude -97.7431 (Austin, TX)."
    )

    print(f"Product: {result.product_name}")
    print(f"Reviewer: {result.reviewer.first_name} {result.reviewer.last_name}")
    print(f"Rating: {'⭐' * result.rating}")
    print(f"Title: {result.title}")
    print(f"Verified: {result.verified_purchase}")
    print(f"Helpful: {result.helpful_count}/{result.total_votes}")
    if result.location:
        print(f"Location: ({result.location.latitude}, {result.location.longitude})")
    print(f"Images: {len(result.images)}")

    return result


def test_with_raw_output():
    """Test with include_raw=True to see the full response"""
    print("\n" + "="*50)
    print("Test 6: With Raw Output")
    print("="*50)

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(Task, include_raw=True)

    result = structured_llm.invoke(
        "Create a simple task: 'Review code' with medium priority, ID 101"
    )

    print(f"Raw message type: {type(result['raw'])}")
    print(f"Parsed task: {result['parsed'].title if result['parsed'] else 'None'}")
    print(f"Parsing error: {result['parsing_error']}")

    if result['raw'].additional_kwargs.get('tool_calls'):
        print(f"Tool calls present: Yes")
        print(f"Tool name: {result['raw'].additional_kwargs['tool_calls'][0]['function']['name']}")

    return result


def test_list_output():
    """Test returning a list of structured objects"""
    print("\n" + "="*50)
    print("Test 7: List of Structured Objects")
    print("="*50)

    class TaskList(BaseModel):
        """Wrapper for multiple tasks using Pydantic v2"""
        model_config = ConfigDict(validate_assignment=True)

        tasks: List[Task] = Field(..., min_length=1, max_length=10)
        total_count: int

        @model_validator(mode='after')
        def validate_count(self) -> Self:
            """Ensure total_count matches the actual number of tasks"""
            if self.total_count != len(self.tasks):
                self.total_count = len(self.tasks)
            return self

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(TaskList)

    result = structured_llm.invoke(
        "Create a list of 3 tasks for a web development project: "
        "1) Set up database (high priority), "
        "2) Create API endpoints (critical priority), "
        "3) Write documentation (low priority)"
    )

    print(f"Total tasks: {result.total_count}")
    for i, task in enumerate(result.tasks, 1):
        print(f"  Task {i}: {task.title} (Priority: {task.priority.value})")

    return result


def test_validation_errors():
    """Test that Pydantic v2 validation works correctly"""
    print("\n" + "="*50)
    print("Test 8: Validation Testing")
    print("="*50)

    # This tests whether the model properly handles validation constraints
    class StrictModel(BaseModel):
        """Model with strict validation rules"""
        model_config = ConfigDict(validate_assignment=True, str_strip_whitespace=True)

        name: Annotated[str, StringConstraints(min_length=3, max_length=20, pattern=r'^[A-Za-z\s]+$')]
        age: conint(ge=18, le=100)
        score: confloat(ge=0.0, le=100.0)
        email: Annotated[str, StringConstraints(pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$')]

    chat_model = get_chat_model()
    structured_llm = chat_model.with_structured_output(StrictModel)

    result = structured_llm.invoke(
        "Create a record for Alice Smith, age 25, score 95.5, email alice@example.com"
    )

    print(f"Name: {result.name}")
    print(f"Age: {result.age}")
    print(f"Score: {result.score}")
    print(f"Email: {result.email}")
    print("✅ Validation passed!")

    return result

In [10]:
test_simple_nested_model()


Test 1: Simple Nested Model (Person)
Type: <class '__main__.Person'>
Name: John Smith
Age: 32
Address: 123 Market St., San Francisco, CA
Contact: john.smith@techcorp.com


Person(first_name='John', last_name='Smith', age=32, address=Address(street='123 Market St.', city='San Francisco', state='CA', zip_code='94105', country='USA'), contact=ContactInfo(email='john.smith@techcorp.com', phone='+14155551234', preferred_contact='email'), nicknames=[], metadata={})

In [11]:
test_task_with_enum()


Test 2: Model with Enum and Optional Fields (Task)
Task ID: 1
Title: Update Authentication System
Priority: high
Assigned to: Jane Doe (jane.doe@example.com)
Subtasks count: 3
  Subtask 1: Update password hashing (Priority: high)
  Subtask 2: Add 2FA support (Priority: critical)
  Subtask 3: Add audit logs (Priority: medium)


Task(id=1, title='Update Authentication System', description=None, priority=<Priority.HIGH: 'high'>, assigned_to=SimplePerson(first_name='Jane', last_name='Doe', email='jane.doe@example.com'), due_date='2024-12-31', tags=[], completed=False, subtasks=[Task(id=101, title='Update password hashing', description=None, priority=<Priority.HIGH: 'high'>, assigned_to=None, due_date=None, tags=[], completed=False, subtasks=[]), Task(id=102, title='Add 2FA support', description=None, priority=<Priority.CRITICAL: 'critical'>, assigned_to=None, due_date=None, tags=[], completed=False, subtasks=[]), Task(id=103, title='Add audit logs', description=None, priority=<Priority.MEDIUM: 'medium'>, assigned_to=None, due_date=None, tags=[], completed=False, subtasks=[])])

In [12]:
test_complex_project()


Test 3: Complex Nested Model (Project)


ValidationError: 4 validation errors for Project
tasks.0.assigned_to
  Input should be a valid dictionary or instance of SimplePerson [type=model_type, input_value='Alice Johnson', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
tasks.1.assigned_to
  Input should be a valid dictionary or instance of SimplePerson [type=model_type, input_value='Bob Chen', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
tasks.2.assigned_to
  Input should be a valid dictionary or instance of SimplePerson [type=model_type, input_value='Carol Williams', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
milestones
  Input should be a valid dictionary [type=dict_type, input_value=[{'name': 'Design Complet..., 'date': '2024-04-01'}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type

In [13]:
test_company_structure()


Test 4: Very Complex Nested Model (Company)


ValidationError: 5 validation errors for Company
name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
founded_year
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
headquarters
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
departments
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
total_employees
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [14]:
test_pydantic_v2_features()


Test 5: Pydantic v2 Features (ProductReview)


ValidationError: 3 validation errors for ProductReview
reviewer.address.zip_code
  String should match pattern '^\d{5}(-\d{4})?$' [type=string_pattern_mismatch, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_pattern_mismatch
reviewer.contact.email
  String should match pattern '^[\w\.-]+@[\w\.-]+\.\w+$' [type=string_pattern_mismatch, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_pattern_mismatch
reviewer.nicknames
  Input should be a valid list [type=list_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type